# Predictor based on population of the county, general health of the population, pharmacies per capita.

In [1]:
# import required libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
scaler =StandardScaler()
import seaborn as sns
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
# read the data needed for the prediction
data_for_prediction = pd.read_csv('Complete_data_for_predictor.csv')
data_for_prediction.head()

,pop_2013,pop_2013_sqmi,median_age,num_households,avg_household_size,obesity_prevalence,diabetes,num_pharmacies
0,300046,173.1,33.0,107332,2.64,69213,16977,51
1,2621,1.1,46.9,1149,2.18,552,152,1
2,32723,11.9,38.0,12187,2.49,6702,1589,7
3,103018,13.9,31.2,36441,2.56,22067,4594,13
4,97147,3.9,34.8,31824,2.75,25287,6372,9


In [4]:
# split columns as predictors and predicted
x = data_for_prediction.drop(['num_pharmacies'], axis=1)
y = pd.DataFrame(data_for_prediction['num_pharmacies'])
# scale the X data
x_scaled = scaler.fit_transform(x)

In [5]:
# split the data
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.2, random_state=20)

In [34]:
# try different models and get their RMSE
# model = LinearRegression()
# model = DecisionTreeRegressor()
# model = GradientBoostingRegressor()

# Run to check the differences
'''
for model_class in (LinearRegression(), DecisionTreeRegressor(), GradientBoostingRegressor()):
    model = model_class
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
'''

RMSE:  31.06935755178896
RMSE:  34.90427726552394
RMSE:  31.18907131380187


/Users/ravirajch/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [47]:
# Start the model training and get the results
model = GradientBoostingRegressor()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('RMSE: ', mean_squared_error(y_test, y_pred, squared=False))
# y_pred_flat = [round(x[0]) for x in y_pred]
y_pred_flat = [round(x) for x in y_pred]
y_test_flat = y_test['num_pharmacies'].tolist()

RMSE:  31.092963350918815


/Users/ravirajch/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [48]:
# calculate the differences and get the result for test dataset
results = pd.DataFrame([y_pred_flat,y_test_flat]).T.rename(columns={0:'predicted',1:'actual'})
results['difference'] = abs(results['predicted']-results['actual'])
results.head()

,predicted,actual,difference
0,4,5,1
1,3,1,2
2,3,4,1
3,2,1,1
4,4,3,1


In [49]:
# get value counts of the values in test set
results[results['difference']<11].difference.value_counts()

1     53
2     34
0     27
3      8
6      8
4      4
5      3
7      3
8      1
10     1
Name: difference, dtype: int64

In [52]:
# repeat the same for the FULL DATA
y_full_preds = model.predict(x_scaled)
# y_full_pred_flat = [round(x[0]) for x in y_pred]
y_full_pred_flat = [round(x) for x in y_full_preds]
y_full_test_flat = y['num_pharmacies'].tolist()
full_results = pd.DataFrame([y_full_pred_flat,y_full_test_flat]).T.rename(columns={0:'predicted',1:'actual'})
full_results['difference'] = abs(full_results['predicted']-full_results['actual'])
full_results.head()

,predicted,actual,difference
0,52,51,1
1,2,1,1
2,7,7,0
3,16,13,3
4,14,9,5


In [53]:
# calculate RMSE
print('RMSE: ', mean_squared_error(y, y_full_preds, squared=False))

RMSE:  15.765159168222038


In [55]:
full_results[full_results['difference']<10].difference.value_counts()

1    284
0    164
2    138
3     59
4     32
5     20
6     18
7     17
8     14
9      5
Name: difference, dtype: int64

In [56]:
full_results.to_csv('ensemble_res.csv', index=False)

In [57]:
full_results.shape

(875, 4)